In [1]:
import numpy as np
import cv2

In [2]:
def forward_transform(img, m):
    indicies_org = np.array(list(np.ndindex(img.shape)), np.uint16)
    hindex = np.hstack([indicies_org, np.ones((len(indicies_org), 1),dtype=np.uint16)]).reshape(len(indicies_org), 3, 1)
    htar = np.float16(m @ hindex)
    htar[:, :2, 0] /= htar[:, 2, [0]]
    indicies_tar = htar[:, :2, 0].astype(np.int32)
    transformed_img = np.zeros_like(img, np.uint8)
    for i, (y, x) in enumerate(indicies_tar):
        if 0 <= y < img.shape[0] and 0 <= x < img.shape[1]:
            transformed_img[y, x] = img[*indicies_org[i]]
    return transformed_img

def backward_transform(img, m):
    indicies_tar = np.array(list(np.ndindex(img.shape)), np.uint16)
    hindex = np.hstack([indicies_tar, np.ones((len(indicies_tar), 1),dtype=np.uint16)]).reshape(len(indicies_tar), 3, 1)
    horg = np.float32(np.linalg.inv(m) @ hindex)
    horg[:, :2, 0] /= horg[:, 2, [0]]
    indicies_org = np.uint16(horg[:, :2, 0])
    transformed_img = np.zeros_like(img, np.uint8)
    for i, (y, x) in enumerate(indicies_tar):
        yorg,xorg = indicies_org[i]
        if 0 <= yorg < img.shape[0] and 0 <= xorg < img.shape[1]:
            transformed_img[y, x] = img[*indicies_org[i]]
    return transformed_img

def calc_perspective_matrix(src, dst):
    A = []
    b = []
    for (x, y), (u, v) in zip(src, dst):
        A.append([x, y, 1, 0, 0, 0, -u*x, -u*y])
        A.append([0, 0, 0, x, y, 1, -v*x, -v*y])
        b.extend([[u],[v]])
    A = np.asarray(A, np.float32)
    b = np.asarray(b, np.float32)
    return np.vstack([np.linalg.solve(A, b), [1]]).reshape(3,3)

In [3]:
img = cv2.imread('GCT-NY.jpg', cv2.IMREAD_GRAYSCALE)
m = np.array([
        [1, 0, 0], # y, instead of x
        [0, 1, 0], # x, instead of y
        [0, 0, 1]
        ])
forward_transformed_img = forward_transform(img, m)
backward_transformed_img = backward_transform(img, m)
cv2.imshow('original', img)
cv2.imshow('forward transformed', forward_transformed_img)
cv2.imshow('backward transformed', backward_transformed_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
points1 = []

def scan(img, points1, points2):
    perspective_matrix = calc_perspective_matrix(points1, points2)
    return cv2.warpPerspective(img, perspective_matrix, points2[2])
    # return backward_transform(img, perspective_matrix) # much slower compared to cv2's method + performance issues

def mouse_callback(event, x, y, flags, param):
    global points1
    if event == cv2.EVENT_LBUTTONDOWN:
        if len(points1) < 4:
            points1.append([x, y])
        print(points1)

In [5]:
scan_switch = False
h = 100
w = 100
img = cv2.imread('NYCTS.jpg', cv2.IMREAD_GRAYSCALE)


while True:
    cv2.imshow('main', img)
    if scan_switch:
        cv2.imshow('scan', scan(img, points1, ((0, 0),(w, 0),(w, h),(0, h))))
    else:
        try:
            cv2.destroyWindow('scan')
        except:
            pass
    cv2.setMouseCallback('main', mouse_callback) # top left -> top right -> bottom left -> bottom right
    key = cv2.waitKey()
    if key == ord('h'):
        if (new_h := h - 10) > 1:
            h = new_h
    if key == ord('H'):
        h += 10
    if key == ord('w'):
        if (new_w := w - 10) > 1:
            w = new_w
    if key == ord('W'):
        w += 10
    if key == ord('s'): # scans
        if len(points1) == 4:
            scan_switch = not scan_switch
    if key == ord('n'): # resets clicked points and exits scan mode
        scan_switch = False
        points1 = []
        try:
            cv2.destroyWindow('scan')
        except:
            pass
        print(points1)
    if key == ord('q'): # quits the program
        break
cv2.destroyAllWindows()